In [ ]:
import pandas as pd
import numpy as np
import mysql.connector as sql
import nltk
from nltk.corpus import stopwords
import random

In [ ]:
try:
    db = sql.connect(host='localhost', database = 'interndb', user='root', password='password')
    query = 'select ID, NPS_Promoter, Response_Text, Response_Text_All from desktop_survey_sample;'
    data = pd.read_sql(query, db)
    db.close()
    print("Connected")
except Exception as e:
    db.close()
    print(str(e))

In [ ]:
data['NPS_Promoter'] = data['NPS_Promoter'].replace(0, 1)

In [ ]:
from langdetect import detect

for i in range(5000):
    if type(data['Response_Text_All'][i]) == int:
        data.replace(to_replace=data['Response_Text_All'][i], value=str(data['Response_Text_All'][i]), inplace=True)


In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)
data['Response_Text_All'] = data.Response_Text_All.apply(lemmatize_text)
data

In [ ]:
rawtokens = list(nltk.word_tokenize(data['Response_Text_All'][int(id) - 1]) for id in data['ID'])
stops = set(stopwords.words('english'))
additional_stops = (',', '.', '/', '?', '<', '>', '!', '@', '#', '$', '%', '^', '&', '*', '//', '\\', ';',"'")
stops.update(additional_stops)
tokens = []
for i in range(data.shape[0]):
    for w in rawtokens[i]:
        if w not in stops:
            tokens.append(w)

In [ ]:
sentiments = [sentiment for sentiment in data['NPS_Promoter']]
documents = [(tokens[i], sentiments[i]) for i in range(data.shape[0])]
random.shuffle(documents)

In [ ]:
all_words = nltk.FreqDist(w.lower() for w in tokens)
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features
all_words

In [ ]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))
len(word_features)

In [ ]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, trainData['NPS_Promoter'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['NPS_Promoter'], prediction_linear, output_dict=True)
print('positive: ', report['1'])
print('negative: ', report['-1'])